In [1]:
import geopandas as gpd
import pandas as pd



In [2]:
blocks_gdf = gpd.read_parquet('data/blocks.parquet')
blocks_gdf.columns

Index(['geometry', 'land_use', 'is_living', 'build_floor_area',
       'living_demand', 'living_area', 'share_living', 'business_area',
       'share_business', 'site_area', 'population', 'footprint_area', 'fsi',
       'gsi', 'l', 'osr', 'mxi', 'capacity_convenience', 'capacity_cafe',
       'capacity_mall', 'capacity_pharmacy', 'capacity_bank', 'capacity_fuel',
       'capacity_pitch', 'capacity_playground', 'capacity_museum',
       'capacity_parking', 'capacity_bakery', 'capacity_restaurant',
       'capacity_dog_park', 'capacity_university', 'capacity_police',
       'capacity_bar', 'capacity_park', 'capacity_government',
       'capacity_supermarket', 'capacity_kindergarten', 'capacity_polyclinic',
       'capacity_school', 'capacity_religion', 'capacity_post',
       'capacity_hairdresser', 'capacity_theatre', 'capacity_stadium',
       'capacity_cemetery', 'capacity_substation', 'capacity_hotel',
       'capacity_hospital', 'capacity_multifunctional_center',
       'capacity_sw

In [3]:
# from blocksnet.analysis.morphotypes import get_strelka_morphotypes

# blocks_df = get_strelka_morphotypes(blocks_gdf)
# blocks_gdf = blocks_gdf.join(blocks_df[['morphotype']], how='left')
# blocks_df.head()

In [4]:
# cadastr_land = gpd.read_parquet('data/spb_land_cad.parquet')
cadastr_buildings = gpd.read_parquet('data/cadastr_buildings_new.parquet')

In [5]:
cadastr_buildings.head()

,cadastralDistrictsCode,floors,building_name,cost_value,cost_index,purpose,year_built,build_record_area,geometry
0,47,17,,1.366803e+09,62696.24,Многоквартирный дом,,21800.4,"POLYGON ((3358160.205 8353937.786, 3358151.752..."
1,47,2,Жилой дом,1.578595e+06,27030.74,Жилое,2022,58.4,"POLYGON ((3414580.283 8393829.825, 3414572.531..."
2,47,2,Здание,1.503600e+06,19105.46,Жилое,2019,78.7,"POLYGON ((3358418.06 8349511.823, 3358412.601 ..."
3,47,3,Жилой дом,4.419648e+06,28943.34,Жилое,2019,152.7,"POLYGON ((3349682.392 8351987.52, 3349685.348 ..."
4,47,1,хозблок,2.197134e+05,13646.79,Нежилое,2022,16.1,"POLYGON ((3413208.648 8393607.511, 3413215.111..."


In [6]:
import numpy as np
import geopandas as gpd

# --- 1. Сбрасываем индексы и создаём новые идентификаторы ---
blocks_gdf = blocks_gdf.reset_index(drop=True).copy()
blocks_gdf['block_idx'] = blocks_gdf.index

cadastr_buildings = cadastr_buildings.reset_index(drop=True).copy()
cadastr_buildings['bldg_idx'] = cadastr_buildings.index

# --- 2. Выбираем только нужные колонки ---
blocks_sub = blocks_gdf[['block_idx', 'geometry']]
buildings_sub = cadastr_buildings[['bldg_idx', 'cost_value', 'cost_index', 'geometry']]

# --- 3. Приводим CRS к единому (если ещё не сделали) ---
buildings_sub = buildings_sub.to_crs(blocks_sub.crs)

# --- 4. Пространственный join по пересечению ---
joined = gpd.sjoin(
    buildings_sub,
    blocks_sub,
    how='inner',
    predicate='intersects'
)
# убедимся, что есть столбцы bldg_idx, cost_value, cost_index, block_idx

# --- 5. Группировка по block_idx ---
agg = (
    joined
    .groupby('block_idx')
    .agg(
        total_cost_value = ('cost_value', 'sum'),
        mean_cost_index  = ('cost_index', 'mean'),
        parcel_count     = ('bldg_idx', 'nunique')
    )
    .reset_index()
)

# --- 6. Сливаем результаты обратно в blocks_gdf ---
blocks_gdf = blocks_gdf.merge(
    agg,
    on='block_idx',
    how='left'
)

# Кварталы без пересечений заполняем нулями
blocks_gdf['total_cost_value'] = blocks_gdf['total_cost_value'].fillna(0)
blocks_gdf['mean_cost_index']  = blocks_gdf['mean_cost_index'].fillna(0)
blocks_gdf['parcel_count']     = blocks_gdf['parcel_count'].fillna(0).astype(int)

# --- 7. Считаем число кварталов с более чем одним пересечением ---
multi_parcel_blocks = (blocks_gdf['parcel_count'] > 1).sum()
print(f'Кварталов с более чем одной пересекающей геометрией: {multi_parcel_blocks}')


Кварталов с более чем одной пересекающей геометрией: 2619


In [7]:
import numpy as np
from scipy import stats

# Вычисляем Z-score
zs = stats.zscore(blocks_gdf['total_cost_value'].fillna(0))
blocks_gdf['z_score'] = zs

# Считаем выбросы как |z| > 3
outliers_z = blocks_gdf[np.abs(blocks_gdf['z_score']) > 3]
print(f'Выбросов по Z-score: {len(outliers_z)}')

Выбросов по Z-score: 186


In [9]:
# Срез на найденные выбросы
outliers_z = blocks_gdf[np.abs(blocks_gdf['z_score']) > 3]

# Первые 10 выбросов
print(outliers_z[['total_cost_value', 'z_score']].head(10))

# Статистика по выбросам
print(outliers_z['total_cost_value'].agg(['min', 'max', 'median', 'mean']))

     total_cost_value    z_score
25       3.520071e+10  10.718715
48       1.290885e+10   3.844230
50       1.094157e+10   3.237549
51       1.235204e+10   3.672518
60       1.269226e+10   3.777437
63       1.185860e+10   3.520350
123      1.134907e+10   3.363219
140      1.350892e+10   4.029283
143      1.092077e+10   3.231138
214      1.800650e+10   5.416270
min       1.024741e+10
max       2.483243e+11
median    1.638224e+10
mean      2.153186e+10
Name: total_cost_value, dtype: float64


In [13]:
blocks_gdf['log_cost'] = np.log1p(blocks_gdf['total_cost_value'])

In [ ]:
blocks_gdf.columns

Index(['geometry', 'land_use', 'is_living', 'build_floor_area',
       'living_demand', 'living_area', 'share_living', 'business_area',
       'share_business', 'site_area', 'population', 'footprint_area', 'fsi',
       'gsi', 'l', 'osr', 'mxi', 'capacity_convenience', 'capacity_cafe',
       'capacity_mall', 'capacity_pharmacy', 'capacity_bank', 'capacity_fuel',
       'capacity_pitch', 'capacity_playground', 'capacity_museum',
       'capacity_parking', 'capacity_bakery', 'capacity_restaurant',
       'capacity_dog_park', 'capacity_university', 'capacity_police',
       'capacity_bar', 'capacity_park', 'capacity_government',
       'capacity_supermarket', 'capacity_kindergarten', 'capacity_polyclinic',
       'capacity_school', 'capacity_religion', 'capacity_post',
       'capacity_hairdresser', 'capacity_theatre', 'capacity_stadium',
       'capacity_cemetery', 'capacity_substation', 'capacity_hotel',
       'capacity_hospital', 'capacity_multifunctional_center',
       'capacity_sw

In [ ]:
from blocksnet.relations import generate_adjacency_graph

graph = generate_adjacency_graph(blocks_gdf, 10)

2025-05-27 18:49:27.750 | INFO     | blocksnet.relations.adjacency.core:_generate_adjacency_nodes:10 - Generating nodes
2025-05-27 18:49:27.755 | INFO     | blocksnet.relations.adjacency.core:_generate_adjacency_edges:15 - Generating edges
2025-05-27 18:49:31.352 | SUCCESS  | blocksnet.relations.adjacency.core:generate_adjacency_graph:38 - Adjacency graph successfully generated: 16320 nodes, 71988 edges


In [ ]:
for block_id in graph.neighbors(3):
    print(block_id)

1
0
306
1058
1059
307
4292
1060
1613
5379


In [ ]:
# Полный скрипт: прогноз изменения стоимости и моделирование пространственного каскада

import geopandas as gpd
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

import libpysal
from spreg import ML_Lag

# 1. Загрузка данных (если ещё не загружено)
# blocks_gdf = gpd.read_file('path_to_your_blocks_data.shp')

# 2. Подготовка признаков и целевой переменной
features = blocks_gdf.drop(columns=['geometry', 'mean_cost_index'])
target   = blocks_gdf['log_cost']

# --- 3. Разделяем числовые и категориальные
num_cols = features.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = features.select_dtypes(exclude=[np.number]).columns.tolist()

df_num = features[num_cols].copy()
df_cat = features[cat_cols].copy()

# --- 4. Импьютация пропусков
df_num = df_num.fillna(df_num.median())         # числовые → медиана
df_cat = df_cat.fillna(df_cat.mode().iloc[0])   # категориальные → мода

# --- 5. One-hot кодирование категориальных
df_cat = pd.get_dummies(df_cat, drop_first=True)

# --- 6. Объединяем обратно в X
X = pd.concat([df_num, df_cat], axis=1)

# --- 7. Масштабирование числовых признаков
scaler = StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])

# --- 8. Приводим все колонки X к float, убираем NaN/inf
X = X.apply(pd.to_numeric, errors='coerce')   # всё в числовой тип
X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(0.0)

# --- 9. Разбиение на train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, target, test_size=0.20, random_state=42
)

# --- 10. RandomForest-регрессор для «базового» сценария
rf = RandomForestRegressor(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)

y_pred_base = rf.predict(X_test)

# --- 11. Сценарий +10% capacity_warehouse
if 'capacity_warehouse' not in X_test.columns:
    raise KeyError("Нет столбца 'capacity_warehouse' в X_test")
X_scenario = X_test.copy()
X_scenario['capacity_warehouse'] *= 1.10
y_pred_scenario = rf.predict(X_scenario)
delta_base = y_pred_scenario - y_pred_base

# --- 12. Строим матрицу Queen-соседства
w = libpysal.weights.Queen.from_dataframe(blocks_gdf)
W = w.full()[0]

# --- 13. Подготавливаем данные для SAR-модели
y_full   = target.astype(float).values.reshape(-1, 1)
X_full   = X.values.astype(float)  # гарантированно numeric

# --- 14. Обучаем SAR (Spatial Lag) модель
sar_model = ML_Lag(
    y_full,
    X_full,
    w=w,
    name_y='log_cost',
    name_x=list(X.columns)
)
print(sar_model.summary)

# --- 15. Вычисляем каскадный отклик (импульс)
rho  = sar_model.rho[0]               # пространственный коэффициент
beta = sar_model.betas.flatten()[1:]  # без константы

I = np.eye(W.shape[0])
M = np.linalg.inv(I - rho * W)

# Выбираем участок j (замените на нужный индекс)
j = 123  
if 'capacity_warehouse' not in X.columns:
    raise KeyError("Нет столбца 'capacity_warehouse' в X")
delta_feat = np.zeros(X.shape[0])
# Учитываем стандартизированное приращение:
orig_value = X.loc[j, 'capacity_warehouse']
delta_feat[j] = orig_value * 0.10

# Локальный эффект
delta_y_local = delta_feat @ beta

# Полный каскадный эффект
delta_y_cascade = M.dot(delta_y_local)

# --- 16. Сохраняем результаты в GeoDataFrame
blocks_gdf['delta_base']    = 0.0
blocks_gdf.loc[X_test.index, 'delta_base'] = delta_base

blocks_gdf['delta_cascade'] = delta_y_cascade

# --- 17. Визуализация карты
ax = blocks_gdf.plot(
    column='delta_cascade',
    cmap='viridis',
    legend=True,
    figsize=(10, 8)
)
ax.set_title('Каскадное изменение log_cost')
ax.set_axis_off()